In [4]:
# みんなのPython第４版 第４章 p.228
%matplotlib inline

import sys
import random
import numpy as np   # numpyはデータサイエンスで頻用されるモジュール
import numpy.random as nprandom   # 一部の乱数生成にnumpyの乱数モジュールを使う

import matplotlib.pyplot as plt
from copy import deepcopy

def around_one(sigma):
    """関数around_one
    平均1で正規分布する非負の擬似乱数を生成する
    引数:
    sigma - 標準偏差
    戻り値:
    平均1，標準偏差sigmaの正規分布に従う乱数の値をひとつ返す。ただし、乱数の値が負である場合には0を返す。
    """
    x = nprandom.randn() * sigma + 1.0
    if x < 0.0:
        return 0.0
    else:
        return x

In [5]:
N_BACTERIA = 10000    # 微生物の初期数

DEFAULT_GROWTH_RATE = 4.0    # 微生物の成長速度のデフォルト値
DEFAULT_CONJUGATION_PROBABILITY = 0.0001    # 微生物の成長速度のデフォルト値
DEFAULT_P_RM = 0.01    # 微生物がある１つの制限修飾系を持っている確率
LOSS_BY_AN_RM = 0.9    # 制限修飾系を１つ持つことによる成長速度の低下

#####

N_PHAGE = 100    # ファージの初期数

DEFAULT_PROPAGATION_RATE = 16.0    # ファージの増殖速度のデフォルト値
DEFAULT_LYSOGENIZE_PROBABILITY = 0.1    # ファージの溶原化確率のデフォルト値

#####

N_BACTERIA_MAX = 1000000    # 微生物の最大数
R_SURVIVE_RESTRICTION = 1.0e-6    # ファージが制限切断を逃れる確率
P_LYSOGEN_WIN = 0.6    # 溶原化したファージが感染ファージに勝つ確率
R_PHAGE_REMAIN = 0.2     # 世代ごとに残留するファージの割合



# みんなのPython第４版 第６章 p.254
class Phage:
    """Phageクラス
    属性:
    r_propagation - 微生物内での複製効率
    p_lysogenize - 溶原化する確率
    modifications - メチル化状態
    host - 溶原化している宿主Bacteriaオブジェクト（溶原化していない時はNone）
    """
    
    # みんなのPython第４版 第６章 p.263 初期化メソッド「__init__()」
        # みんなのPython第４版 第４章 p.152 メソッドとは
    # みんなのPython第４版 第６章 p.274 第１引数self
    def __init__(self, strain, r_propagation, p_lysogenize, modifications):
        """Phageクラスオブジェクトを初期化
        引数:
        strain - 系統名（ユニークな文字列）
        r_propagation - 微生物内での複製効率
        p_lysogenize - 溶原化する確率
        modifications - メチル化状態
        """
        self.strain = strain
        self.r_propagation = r_propagation
        self.p_lysogenize = p_lysogenize
        self.modifications = modifications
        self.lysogenized_host = None
        self.infected_host = None
        self.killed = False
    
    def induction(self):
        """溶菌サイクルに入るかどうかを決める
        戻り値:
        True - 溶菌サイクルに入る
        False - 溶原化状態を維持
        ＊＊＊＊＊グループワークで自由に書きかえてよい＊＊＊＊＊
        """
        if self.lysogenized_host.infection is not None:
            return True
        elif nprandom.random() <= 0.1:
            return True
        else:
            return False

class Bacteria:
    """Bacteriaクラス"""
    def __init__(self, r_growth, p_conjugation, rm_systems, strain = None):
        """Bacteriaクラスオブジェクトを初期化
        引数:
        """
        self.strain = strain
        self.r_growth = r_growth
        self.p_conjugation = p_conjugation
        self.rm_systems = rm_systems
        self.lysogen = None
        self.infection = None
        self.killed = False




In [6]:
# 細菌の集団を表現するリスト。要素はBacteriaクラスのオブジェクト
# みんなのPython第４版 第６章 p.254 オブジェクトとクラス
bacteria_population = []

def init_r_growth():
    """成長速度の初期化"""
    return DEFAULT_GROWTH_RATE * around_one(0.02)

def init_p_conjugation():
    """接合確率の初期化"""
    return DEFAULT_CONJUGATION_PROBABILITY * around_one(0.05)

def init_rm():
    """制限修飾系リストの初期化
    64種類の制限修飾系のひとつひとつについて、DEFAULT_P_RMの確率で
    保有しているかどうかを決め、保有している場合は1、していない場合は0
    を値とする要素64個のリストを返す。
    戻り値:
    要素64個のリスト。各要素の値は1または0
    """
    rm = []
    for i in range(64):
        if nprandom.random() <= DEFAULT_P_RM:
            rm.append(1)
        else:
            rm.append(0)
    return rm

# 細菌の集団をつくる。初期の個体数はN_BACTERIA
for i in range(N_BACTERIA):
    bacteria_population.append( Bacteria( init_r_growth(), init_p_conjugation(), init_rm() ))
    """Bacteria( init_r_growth(), init_p_conjugation(), init_rm() ) で、
    Bacteria.__init__()で初期化されたBacteriaオブジェクトがつくられる。
    メソッド__init__()の第１引数selfは省略するので、与えた引数は第２〜４引数になることに注意。
    """

# 細菌集団の内容を一部確認してみる
print len(bacteria_population)
print bacteria_population[-1].r_growth
print bacteria_population[-1].p_conjugation
print bacteria_population[-1].rm_systems



10000
3.94781531245
0.000104413493834
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [7]:
# ファージの集団を表現するリスト。要素はBacteriaクラスのオブジェクト
phage_population = []

def init_phage_strain_name(i):
    """ファージ系統名の初期化
    ＊＊＊＊＊グループワークで自由に書きかえてよい＊＊＊＊＊
"""
    return str(i)

def init_r_propagation():
    """増殖速度の初期化
    ＊＊＊＊＊グループワークで自由に書きかえてよい＊＊＊＊＊
    """
    return DEFAULT_PROPAGATION_RATE * around_one(0.1)

def init_p_lysogenize():
    """溶原化速度の初期化
    ＊＊＊＊＊グループワークで自由に書きかえてよい＊＊＊＊＊
"""
    return DEFAULT_LYSOGENIZE_PROBABILITY * around_one(0.1)

def init_modifications():
    """修飾状態の初期化
    細菌集団bacteria_populationからランダムに１個体の「親」を選び、
    その個体が持つ制限修飾系によって修飾（メチル化）されているものとする。
    （ファージが存在するということは、親の細菌は溶菌で死んだと考えるなら、
    こうしたアルゴリズムでなく、ランダムに決めるというモデリングもありうる）
    """
    return random.choice(bacteria_population).rm_systems


# ファージの集団をつくる。初期の個体数はN_PHAGE
for i in range(N_PHAGE):
    phage_population.append( Phage( init_phage_strain_name(i), 
                                    init_r_propagation(), 
                                    init_p_lysogenize(), 
                                    init_modifications() ))

# ファージ集団の内容を一部確認してみる
print len(phage_population)
print phage_population[-1].r_propagation
print phage_population[-1].p_lysogenize
print phage_population[-1].modifications


100
15.9038295601
0.107829325818
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [8]:
def update_populations(pp = phage_population, bp = bacteria_population, shuffle = False):
    """ファージ、微生物の集団から死滅した個体を除去する
    引数:
    pp - ファージ集団（Phageオブジェクトのリスト）
    bp - 細菌集団（Bacteriaオブジェクトのリスト）
    shuffle - シャッフルの有無
    戻り値:
    なし
    pp,bpのそれぞれから属性killedがTrueのオブジェクトを取り除き、生存個体だけの集団に行進する。
    shuffleがTrueの場合、pp,bpそれぞれのリスト内のオブジェクトの並び順をランダムにシャッフルする。
    """
    global phage_population, bacteria_population
    # 右辺はリスト内包表記 みんなのPython第４版 第５章 p.235 
    phage_population = [ p for p in phage_population if p.killed is False ]
    bacteria_population = [ b for b in bacteria_population if b.killed is False ]
    if shuffle:
        random.shuffle(phage_population)
        random.shuffle(bacteria_population)

def p_survive_restriction(phage):
    """ファージが微生物の制限切断を生き抜く確率を返す
    引数:
    phage - Phageオブジェクト
    戻り値:
    phageが、感染した細菌（phage.infected_host）の制限修飾系による
    免疫に対して生存する確率
    """
    n = list((np.array(phage.modifications) - 1) * np.array(phage.infected_host.rm_systems)).count(-1)
    if n == 0:
        return 1.0
    else:
        return pow(R_SURVIVE_RESTRICTION,n)

def induce_lytic_cycle(phage):
    """溶原化したファージが溶菌サイクルに入って増殖し、溶菌する
    引数:
    phage - Phageオブジェクト
    戻り値:なし
    溶原化していたphageが、宿主細菌（phage.lysogenized_host）を殺し
    自由生活するファージとして増殖する。
    """
    phage.lysogenized_host.killed = True
    phage.lysogenized_host = None
    for i in range( int(phage.r_propagation * around_one(0.1)) ):
        phage_population.append( deepcopy(phage) )

def lysis(phage):
    """感染したファージが増殖して溶菌する
    引数:
    phage - Phageオブジェクト
    戻り値:なし
    感染したphageが、感染した細菌（phage.infected_host）を殺し
    自由生活するファージとして増殖する。
    """
    if phage.infected_host.lysogen is not None:
        phage.infected_host.lysogen.killed = True
    phage.infected_host.killed = True
    phage.infected_host = None
    for i in range( int(phage.r_propagation * around_one(0.1)) ):
        phage_population.append( deepcopy(phage) )

def invoke_a_generation():
    """１世代進める"""
    global phage_population, bacteria_population

    update_populations(shuffle = True)
    
    # ファージが感染する個体を決定
    # すでに感染者がいる菌に当たった場合は、感染できず、この世代は「お休み」
    for p in [ pg for pg in phage_population if pg.lysogenized_host is None ]:
        b = random.choice(bacteria_population)
        if b.infection is None:
            p.infected_host = b
            b.infection = p

    # 微生物によるファージの制限切断
    for p in [ ip for ip in phage_population if ip.infected_host is not None ]:
        # 制限切断の有無を判定
        if nprandom.random() > p_survive_restriction(p):
            p.killed = True
            p.infected_host.infection = None
        else:
            # 溶原化・溶菌にかかわらず、宿主の修飾を受ける
            p.modifications = p.infected_host.rm_systems

    update_populations()

    # 溶原化ファージの反応
    for b in [ lb for lb in bacteria_population if lb.lysogen is not None ]:
        if b.lysogen.induction():
            if b.infection == None or nprandom.random() < P_LYSOGEN_WIN:
                induce_lytic_cycle(b.lysogen)
            else:
                lysis(b.infection)
        else:
            if b.infection is not None:
                if nprandom.random() <= b.infection.p_lysogenize:
                    # 先客がいるため、溶原化に失敗する
                    b.infection.killed = True
                    b.infection = None
                else:
                    lysis(b.infection)

    update_populations()

    # 溶原化ファージのない微生物に感染したファージの転帰確定
    for b in [ bac for bac in bacteria_population if (bac.lysogen is None) and (bac.infection is not None) ]:
        if nprandom.random() <= b.infection.p_lysogenize:
            b.lysogen = b.infection
            b.infection = None
            b.lysogen.lysogenized_host = b
            b.lysogen.infected_host = None
        else:
            lysis(b.infection)

    update_populations()

    # 微生物の増殖
    for b in bacteria_population:
        for i in range( int(b.r_growth * pow(LOSS_BY_AN_RM, len(b.rm_systems)) * around_one(0.1)) ):
            bacteria_population.append( deepcopy(b) )
    
    if len(bacteria_population) > N_BACTERIA_MAX:
        random.shuffle(bacteria_population)
        bacteria_population = bacteria_population[:N_BACTERIA_MAX]
    
    # ファージの喪失
    if len(phage_population):
        phage_population = phage_population[ : int(len(phage_population) * R_PHAGE_REMAIN) ]


In [9]:
for i in range(20):
    invoke_a_generation()
    print "=== Generation {} ===".format(i)
    print len(phage_population)
    print len(bacteria_population)
    if len(phage_population) == 0 or len(bacteria_population) == 0:
        break


=== Generation 0 ===
177
9946
=== Generation 1 ===
275
9863
=== Generation 2 ===
418
9740
=== Generation 3 ===
558
9576
=== Generation 4 ===
737
9361
=== Generation 5 ===
984
9075
=== Generation 6 ===
1276
8703
=== Generation 7 ===
1528
8263
=== Generation 8 ===
1702
7781
=== Generation 9 ===
1729
7293
=== Generation 10 ===
1534
6864
=== Generation 11 ===
1212
6532
=== Generation 12 ===
904
6281
=== Generation 13 ===
604
6118
=== Generation 14 ===
382
6017
=== Generation 15 ===
231
5957
=== Generation 16 ===
127
5927
=== Generation 17 ===
66
5913
=== Generation 18 ===
62
5897
=== Generation 19 ===
37
5888
